# Term Frequency-Inverse Document Frequency

This feature will compare the frequencies of n-grams used in the comment to those used in the article. We could also augment this feature with WordNet to compare the frequencies of related words in both documents

The term will be defined as the n-grams in the comment. Term frequency will come from the frequency of a given term in the document that the comment is referring to. Inverse document frequency will be defined as the frequency of the term in all of the other articles that we have scraped. This could prove to be too much for a computer to perform in a reasonable amount of time. If this is the case, we can compare it to 10 other random articles or something like that

Going to first code a quick example of how TFIDF should work. The documents used and results garnered should match the wikipedia page for tfidf which can be found here: https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [2]:
from collections import Counter
import math

In [9]:
doc1 = 'this is a a sample'
doc2 = 'this is another another example example example'
corpus = [doc1, doc2]

In [10]:
def tf(term, document):
    term_arr = str.split(document)
    term_dict = Counter(term_arr)
    return term_dict[term] / len(term_arr)

In [11]:
def idf(term, corpus):
    numerator = len(corpus)
    count = 0
    for doc in corpus:
        term_arr = str.split(doc)
        if term in term_arr:
            count += 1
    if count == 0:
        return 0
    return math.log10(numerator/count)


In [12]:
def tfidf(term, document, corpus):
    return tf(term, document) * idf(term, corpus)

In [13]:
tfidf('sample', doc1, corpus)

0.06020599913279624

I think I can use this same exact code on actual reddit data. The term will be each word in a comment. Loop through every word, grab the tfidf score for each word in the comment, and sum these scores together to get the comment's tfidf score. Document is pure article text (need to look at what Sam did for word score comparisons to grab article text), corpus is just a list of the document variables (which, again, are just pure text)

In [14]:
import pandas as pd
topics_data = pd.read_csv('files/compiled_topics.csv')
comments_data = pd.read_csv('files/compiled_comments.csv')

In [15]:
random_topics_sample = topics_data.sample(n=3)

In [16]:
random_topics_sample

,title,score,id,url,comms_num,created,body,text
379,Democratic candidates for Baltimore mayor spen...,77,ijxzxa,https://apnews.com/24fada4cc4c1ebd8370f60caa74...,38,1.598909e+09,NaN,baltimore (ap) — the top six democratic candid...
1356,Are immigrants in US detention centers free to...,781,c8tsin,https://www.reddit.com/r/NeutralPolitics/comme...,339,1.562217e+09,Conditions are very poor in U.S. immigration d...,Conditions are very poor in U.S. immigration d...
724,Attorney General William Barr to depart admini...,53,kd9801,https://www.nbcnews.com/politics/politics-news...,17,1.608017e+09,NaN,Attorney General William Barr will leave his p...


In [17]:
random_topics_sample['score'].mean()

303.6666666666667

In [18]:
#This code takes a bit too long...might be better to take mean instead of min (which has already been seen to run faster),
#lower the minimum requirement, or find a better way to sample with conditions instead of randomly sampling everytime
while True:
    random_topics_sample = topics_data.sample(n=10)
    if ((random_topics_sample['score'].min() > 100) and (random_topics_sample['comms_num'].min() > 50)):
        break
random_topics_sample

,title,score,id,url,comms_num,created,body,text
1318,Does the federal government possess the power ...,338,cr5yg0,https://www.reddit.com/r/NeutralPolitics/comme...,374,1.565989e+09,"Yesterday, [Beto O'rourke](https://www.huffpos...","Yesterday, Beto O'rourke came out in favor of ..."
1275,What are the pros and cons of the US withdrawi...,505,dfxd42,https://www.reddit.com/r/NeutralPolitics/comme...,92,1.570740e+09,What is the Open Skies Treaty? The treaty allo...,What is the Open Skies Treaty? The treaty allo...
233,How Local Covid Deaths Are Affecting Vote Choice,155,hzevob,https://www.nytimes.com/2020/07/28/upshot/poll...,56,1.595973e+09,NaN,to understand whether these community-level ex...
75,Hannity Says It’s ‘Despicable’ to Call for Pol...,479,bxtj3y,https://www.thedailybeast.com/hannity-says-its...,106,1.559936e+09,NaN,"apparently, “lock her up!” never happened.\n\n..."
70,Durbin: McConnell ignored election security be...,133,blxd09,https://www.politico.com/story/2019/05/07/dick...,74,1.557298e+09,NaN,"""he ignores the mueller report and our intelli..."
1313,Can the President Order Firms to Leave China?,557,cvbghy,https://www.reddit.com/r/NeutralPolitics/comme...,187,1.566783e+09,US President Donald Trump [tweeted](https://tw...,US President Donald Trump tweeted on Friday th...
1004,What is the evidence supporting and refuting t...,838,kaubu5,https://www.reddit.com/r/NeutralPolitics/comme...,460,1.607685e+09,[Donald Trump has requested](https://www.cnn.c...,Donald Trump has requested the Supreme Court o...
79,Donald Trump Jr. shared a racist tweet about K...,301,c75b2i,https://www.businessinsider.com/donald-trump-j...,89,1.561875e+09,[deleted],"donald trump jr., the president's eldest son a..."
147,Supreme Court Rules State 'Faithless Elector' ...,212,hm8qvv,https://www.npr.org/2020/07/06/885168480/supre...,77,1.594075e+09,NaN,supreme court rules state 'faithless elector' ...
209,Trump consults Bush torture lawyer on how to s...,294,hv6rn4,https://www.theguardian.com/us-news/2020/jul/2...,69,1.595365e+09,NaN,the trump administration has been consulting t...


In [19]:
subIds = random_topics_sample['id'].unique().tolist()
subIds

['cr5yg0',
 'dfxd42',
 'hzevob',
 'bxtj3y',
 'blxd09',
 'cvbghy',
 'kaubu5',
 'c75b2i',
 'hm8qvv',
 'hv6rn4']

In [21]:
random_comments_sample = pd.DataFrame(columns = comments_data.columns)
for i in subIds:
    frame = comments_data[comments_data['submissionId'] == i]
    frames = [random_comments_sample, frame]
    random_comments_sample = pd.concat(frames)

random_comments_sample

,action,content,author,details,submissionId,commentId,WordScore,WholeScore
10734,NaN,[/r/NeutralPolitics](https://www.reddit.com/r/...,tkc80,NaN,cr5yg0,ex37105,0.109078,0.771502
10735,NaN,"I'm going to assume that by ""assault weapon"" y...",NaN,NaN,cr5yg0,ex3d7io,1.120470,0.839213
10736,NaN,"u/Chasicle and others have asked ""What is an a...",prime_23571113,NaN,cr5yg0,ex3ki3m,1.218901,0.863967
10737,NaN,"Robert Francis ""Beto"" O'Rourke and other Democ...",no112358,NaN,cr5yg0,ex5ps5h,0.826000,0.778125
10738,NaN,The biggest problem you're going to have with ...,tklite,NaN,cr5yg0,ex3c04q,0.437754,0.861470
...,...,...,...,...,...,...,...,...
2843,removecomment,If people would start actually paying attentio...,dbb507dc2ef55cb83730e2a5e44a4805,remove,hv6rn4,5f1782be4af4320009455033,1.190177,0.884120
2844,approvecomment,I wouldnâ€™t say itâ€™s what aboutism. I would...,c5fdabdbfaa7e7a31c48eb11cd31c1eb,confirm_ham,hv6rn4,5f1785084af432000945503d,0.000611,0.868083
2875,removecomment,l,ae02c4ee90ec3927e8d113655b927e94,One-word response,hv6rn4,5f1876704af43200094558c2,0.000000,0.863451
3673,approvecomment,"And yet, these are the same people that preten...",e12d2abe6ac18532fc24fd610695a31c,confirm_ham,hv6rn4,5f2ae7eeee857d0009966cac,0.350250,0.891546


In [22]:
random_comments_sample['submissionId'].unique()

array(['cr5yg0', 'dfxd42', 'hzevob', 'bxtj3y', 'blxd09', 'cvbghy',
       'kaubu5', 'c75b2i', 'hm8qvv', 'hv6rn4'], dtype=object)

In [23]:
random_topics_sample['id'].unique()

array(['cr5yg0', 'dfxd42', 'hzevob', 'bxtj3y', 'blxd09', 'cvbghy',
       'kaubu5', 'c75b2i', 'hm8qvv', 'hv6rn4'], dtype=object)

In [25]:
random_comments_sample.to_csv("files/random_comments_sample216.csv", index=False)
random_topics_sample.to_csv("files/random_topics_sample216.csv", index=False)

### To avoid having to read in more data, feel free to start running code from here onward
My plan is to first run the program and see if it works on this small sample size and then pull in the large dataframe and see how that goes

In [57]:
random_comments_sample = pd.read_csv("files/random_comments_sample216.csv")
random_topics_sample = pd.read_csv("files/random_topics_sample216.csv")

import numpy as np
random_comments_sample['tfidf'] = np.nan

In [58]:
#Remember that sometimes the url is actually just a link back to the main reddit thread and in that case we will be
#performing tfidf on whatever the first user posted against comments, not articles

In [66]:
#Stuck on trying to figure out the best way to add the tfidf term to the dataframe...seems like it will be wildly
#inefficient to loop through anything
corpus = []
id_list = []
for index, topic in random_topics_sample.iterrows():
    corpus.append(topic['text'])
    id_list.append(topic['id'])
    
tfidf_list = []
    
for index, comment in random_comments_sample.iterrows():
    tfidf_sum = 0
    tfidf_current = 0
    doc = random_topics_sample[random_topics_sample['id'] == comment['submissionId']]['text']
    doc = str(doc)
    term = comment['content']
    term_arr = str.split(term)
    for element in term_arr:
        tfidf_current = tfidf(element, doc, corpus)
        tfidf_sum += tfidf_current
    tfidf_list.append(tfidf_sum)
    
    
random_comments_sample['tfidf'] = tfidf_list

##### The above code works for the random sampling of comments but it already takes a bit of time. Now I'm going to try to make it work on the full compiled comments and compiled topics data frame as well as turn it into a function

In [18]:
import numpy as np
import pandas as pd
from collections import Counter
import math

def tf(term, document):
    document = str(document)
    term_arr = str.split(document)
    term_dict = Counter(term_arr)
    return term_dict[term] / len(term_arr)

def idf(term, corpus):
    numerator = len(corpus)
    count = 0
    for doc in corpus:
        doc = str(doc)
        term_arr = str.split(doc)
        if term in term_arr:
            count += 1
    if count == 0:
        return 0
    return math.log10(numerator/count)

def tfidf(term, document, corpus):
    return tf(term, document) * idf(term, corpus)

def tfidf_on_dataset(topics_df, comments_df):
    corpus = []
    id_list = []
    for index, topic in topics_df.iterrows():
        corpus.append(topic['text'])
        id_list.append(topic['id'])
    
    tfidf_list = []
    
    for index, comment in comments_df.iterrows():
        tfidf_sum = 0
        tfidf_current = 0
        doc = topics_df[topics_df['id'] == comment['submissionId']]['text']
        doc = str(doc)
        term = comment['content']
        term = str(term)
        term_arr = str.split(term)
        for element in term_arr:
            tfidf_current = tfidf(element, doc, corpus)
            tfidf_sum += tfidf_current
        tfidf_list.append(tfidf_sum)
        print(str((index/(len(comments_df.index)))*100) + "% complete")
    return tfidf_list

#### The above code will work on a full dataset...but I'm not sure how long it would take to run, so let's try it

In [19]:
full_topics_df = pd.read_csv("files/compiled_topics.csv")
full_comments_df = pd.read_csv("files/compiled_comments_3_14_2021.csv")
full_comments_df = full_comments_df.drop(['Unnamed: 0'], axis=1)
full_comments_df

,action,content,author,details,submissionId,commentId,WordScore,WholeScore,tfidf,contains_url,...,contains_!,no_url_WordScore,no_url_WholeScore,WordScoreNoStop,WholeScoreNoStop,no_url_or_stops_WholeScore,no_url_or_stops_WordScore,no_url_or_stops_content,NER_count,NER_match
0,True,So what are the implications here? Does it onl...,Cody_Fox23,NaN,4op948,d4eictg,0.000000,0.849655,0.001573,False,...,False,0.000000,0.816813,0.000000,0.773069,0.736582,0.000000,So implications here? Does affect involved Vis...,0,0
1,True,Sadly this isn't new. Police officers use many...,DrFrenchman,NaN,4sef35,d58ts90,0.000000,0.900283,0.255802,False,...,True,0.000000,0.884829,0.000000,0.857654,0.844658,0.000000,Sadly isn't new. Police officers use faulty te...,0,0
2,True,What's disturbing about this is that our gover...,bbakks,NaN,4sef35,d58y081,-0.038865,0.869078,0.000000,False,...,False,-0.038865,0.866455,-0.038865,0.833865,0.785302,-0.038865,What's disturbing government destroying lives ...,1,0
3,True,What I find really concerning is the horrible ...,poliscijunki,NaN,4sef35,d5919n8,0.000000,0.898426,0.000000,True,...,False,0.000000,0.884435,0.000000,0.865826,0.852412,0.000000,What I find concerning horrible response law e...,1,0
4,True,This subject might have legs but this article ...,interweb1,NaN,64zsim,dg6l969,0.000000,0.850127,0.000000,False,...,False,0.000000,0.835723,0.000000,0.826162,0.804306,0.000000,This subject legs article opinion piece editor...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,True,"Yes, while in East Baghdad my platoons mission...",CapitalCockroach,NaN,bav0rl,ekggrgk,1.070477,0.840028,0.000000,True,...,False,1.070477,0.831097,1.000532,0.827872,0.788655,1.000532,"Yes, East Baghdad platoons mission check build...",3,0
10232,True,The [definition the FBI currently uses for int...,CQME,NaN,bav0rl,ekyelps,0.941533,0.882768,0.217543,True,...,False,0.884132,0.870870,0.606157,0.852373,0.843292,0.600762,The [definition FBI currently uses internation...,3,0
10233,True,[Yes.](https://en.m.wikipedia.org/wiki/Islamic...,Silent_As_The_Grave_,NaN,bav0rl,ekehcqg,0.217683,0.779386,0.000000,True,...,False,0.217683,0.833056,0.369440,0.782545,0.800717,0.369440,[Yes.] Have look allies with. Hezbollah fucks ...,1,0
10234,True,Has ANY Shia ever committed an act of terroris...,bsmdphdjd,NaN,bav0rl,ekfp4ls,1.293729,0.861529,0.000000,False,...,False,1.293729,0.847163,1.788263,0.834425,0.792615,1.788263,Has ANY Shia committed act terrorism U.S.?\n\n...,4,0


In [20]:
def df(topics_df, comments_df):
    column_names = ['term', 'idf']
    df_df = pd.DataFrame(columns = column_names)
    
    corpus = []
    for index, topic in topics_df.iterrows():
        corpus.append(topic['text'])
    
    for index, comment in comments_df.iterrows():
        term = comment['content']
        term = str(term)
        term_arr = str.split(term)
        for element in term_arr:
            element = str(element)
            if element in df_df.values:
                continue
            else:
                df_val = idf(element, corpus)
                df2 = pd.DataFrame([[element, df_val]], columns = column_names)
                df_df = df_df.append(df2)
        print(str((index/(len(comments_df.index)))*100) + "% complete")
    df_df.to_csv("files/tfidf_ss.csv", index=False)
    
df(full_topics_df, full_comments_df)
            

0.0% complete
0.009769441187964049% complete
0.019538882375928098% complete
0.029308323563892142% complete
0.039077764751856196% complete
0.04884720593982024% complete
0.058616647127784284% complete
0.06838608831574833% complete
0.07815552950371239% complete
0.08792497069167644% complete
0.09769441187964048% complete
0.10746385306760453% complete
0.11723329425556857% complete
0.12700273544353263% complete
0.13677217663149666% complete
0.14654161781946073% complete
0.15631105900742479% complete
0.16608050019538884% complete
0.1758499413833529% complete
0.1856193825713169% complete
0.19538882375928096% complete
0.20515826494724504% complete
0.21492770613520906% complete
0.2246971473231731% complete
0.23446658851113714% complete
0.24423602969910121% complete
0.25400547088706527% complete
0.2637749120750293% complete
0.2735443532629933% complete
0.2833137944509574% complete
0.29308323563892147% complete
0.3028526768268855% complete
0.31262211801484957% complete
0.3223915592028136% complete

2.7842907385697537% complete
2.7940601797577176% complete
2.803829620945682% complete
2.813599062133646% complete
2.82336850332161% complete
2.8331379445095743% complete
2.842907385697538% complete
2.852676826885502% complete
2.8624462680734664% complete
2.87221570926143% complete
2.8819851504493945% complete
2.8917545916373584% complete
2.9015240328253222% complete
2.9112934740132865% complete
2.9210629152012504% complete
2.9308323563892147% complete
2.9406017975771785% complete
2.9503712387651424% complete
2.9601406799531067% complete
2.9699101211410706% complete
2.979679562329035% complete
2.9894490035169987% complete
2.999218444704963% complete
3.008987885892927% complete
3.0187573270808907% complete
3.028526768268855% complete
3.038296209456819% complete
3.048065650644783% complete
3.057835091832747% complete
3.067604533020711% complete
3.077373974208675% complete
3.087143415396639% complete
3.0969128565846034% complete
3.1066822977725677% complete
3.1164517389605315% complete
3.1

5.627198124267292% complete
5.636967565455256% complete
5.64673700664322% complete
5.656506447831184% complete
5.666275889019149% complete
5.676045330207112% complete
5.685814771395076% complete
5.69558421258304% complete
5.705353653771004% complete
5.715123094958968% complete
5.724892536146933% complete
5.734661977334897% complete
5.74443141852286% complete
5.754200859710824% complete
5.763970300898789% complete
5.773739742086752% complete
5.783509183274717% complete
5.793278624462681% complete
5.8030480656506445% complete
5.812817506838608% complete
5.822586948026573% complete
5.832356389214537% complete
5.842125830402501% complete
5.851895271590465% complete
5.861664712778429% complete
5.871434153966392% complete
5.881203595154357% complete
5.890973036342322% complete
5.900742477530285% complete
5.9105119187182495% complete
5.920281359906213% complete
5.930050801094177% complete
5.939820242282141% complete
5.949589683470106% complete
5.95935912465807% complete
5.969128565846034% com

8.489644392340757% complete
8.499413833528722% complete
8.509183274716687% complete
8.51895271590465% complete
8.528722157092613% complete
8.53849159828058% complete
8.548261039468542% complete
8.558030480656505% complete
8.567799921844472% complete
8.577569363032435% complete
8.587338804220398% complete
8.597108245408362% complete
8.606877686596327% complete
8.61664712778429% complete
8.626416568972255% complete
8.636186010160218% complete
8.645955451348183% complete
8.655724892536147% complete
8.66549433372411% complete
8.675263774912075% complete
8.68503321610004% complete
8.694802657288003% complete
8.704572098475968% complete
8.71434153966393% complete
8.724110980851895% complete
8.73388042203986% complete
8.743649863227823% complete
8.753419304415786% complete
8.763188745603752% complete
8.772958186791715% complete
8.782727627979678% complete
8.792497069167643% complete
8.802266510355608% complete
8.812035951543571% complete
8.821805392731536% complete
8.8315748339195% complete
8

11.381398983978116% complete
11.39116842516608% complete
11.400937866354043% complete
11.410707307542008% complete
11.420476748729973% complete
11.430246189917936% complete
11.4400156311059% complete
11.449785072293865% complete
11.459554513481828% complete
11.469323954669793% complete
11.479093395857758% complete
11.48886283704572% complete
11.498632278233686% complete
11.508401719421649% complete
11.518171160609613% complete
11.527940601797578% complete
11.537710042985541% complete
11.547479484173504% complete
11.55724892536147% complete
11.567018366549433% complete
11.576787807737396% complete
11.586557248925361% complete
11.596326690113326% complete
11.606096131301289% complete
11.615865572489254% complete
11.625635013677217% complete
11.635404454865181% complete
11.645173896053146% complete
11.654943337241109% complete
11.664712778429074% complete
11.674482219617039% complete
11.684251660805002% complete
11.694021101992966% complete
11.70379054318093% complete
11.713559984368894% 

...

88.34505666275889% complete
88.35482610394686% complete
88.36459554513482% complete
88.37436498632279% complete
88.38413442751074% complete
88.39390386869871% complete
88.40367330988667% complete
88.41344275107463% complete
88.4232121922626% complete
88.43298163345057% complete
88.44275107463852% complete
88.45252051582649% complete
88.46228995701446% complete
88.47205939820242% complete
88.48182883939039% complete
88.49159828057836% complete
88.50136772176631% complete
88.51113716295428% complete
88.52090660414224% complete
88.53067604533021% complete
88.54044548651817% complete
88.55021492770614% complete
88.55998436889409% complete
88.56975381008206% complete
88.57952325127003% complete
88.58929269245799% complete
88.59906213364596% complete
88.60883157483393% complete
88.61860101602188% complete
88.62837045720985% complete
88.63813989839781% complete
88.64790933958577% complete
88.65767878077374% complete
88.6674482219617% complete
88.67721766314966% complete
88.68698710433763% com

91.22704181320829% complete
91.23681125439624% complete
91.24658069558421% complete
91.25635013677218% complete
91.26611957796014% complete
91.27588901914811% complete
91.28565846033607% complete
91.29542790152402% complete
91.30519734271199% complete
91.31496678389996% complete
91.32473622508792% complete
91.33450566627589% complete
91.34427510746386% complete
91.35404454865181% complete
91.36381398983978% complete
91.37358343102775% complete
91.38335287221571% complete
91.39312231340368% complete
91.40289175459164% complete
91.4126611957796% complete
91.42243063696756% complete
91.43220007815553% complete
91.44196951934349% complete
91.45173896053146% complete
91.46150840171943% complete
91.47127784290738% complete
91.48104728409535% complete
91.49081672528332% complete
91.50058616647128% complete
91.51035560765925% complete
91.5201250488472% complete
91.52989449003516% complete
91.53966393122313% complete
91.54943337241109% complete
91.55920281359906% complete
91.56897225478703% com

94.10902696365768% complete
94.11879640484564% complete
94.12856584603361% complete
94.13833528722158% complete
94.14810472840954% complete
94.1578741695975% complete
94.16764361078546% complete
94.17741305197342% complete
94.18718249316139% complete
94.19695193434936% complete
94.20672137553731% complete
94.21649081672528% complete
94.22626025791325% complete
94.23602969910121% complete
94.24579914028918% complete
94.25556858147715% complete
94.2653380226651% complete
94.27510746385308% complete
94.28487690504103% complete
94.294646346229% complete
94.30441578741696% complete
94.31418522860491% complete
94.32395466979288% complete
94.33372411098085% complete
94.34349355216881% complete
94.35326299335678% complete
94.36303243454475% complete
94.3728018757327% complete
94.38257131692068% complete
94.39234075810865% complete
94.4021101992966% complete
94.41187964048456% complete
94.42164908167253% complete
94.43141852286048% complete
94.44118796404845% complete
94.45095740523642% complet

96.99101211410708% complete
97.00078155529503% complete
97.010550996483% complete
97.02032043767097% complete
97.03008987885893% complete
97.0398593200469% complete
97.04962876123486% complete
97.05939820242281% complete
97.06916764361078% complete
97.07893708479874% complete
97.08870652598671% complete
97.09847596717468% complete
97.10824540836263% complete
97.1180148495506% complete
97.12778429073857% complete
97.13755373192653% complete
97.1473231731145% complete
97.15709261430247% complete
97.16686205549043% complete
97.1766314966784% complete
97.18640093786635% complete
97.19617037905431% complete
97.20593982024228% complete
97.21570926143025% complete
97.2254787026182% complete
97.23524814380617% complete
97.24501758499414% complete
97.2547870261821% complete
97.26455646737007% complete
97.27432590855804% complete
97.284095349746% complete
97.29386479093395% complete
97.30363423212192% complete
97.31340367330988% complete
97.32317311449785% complete
97.33294255568582% complete
97

99.87299726455646% complete
99.88276670574443% complete
99.8925361469324% complete
99.90230558812036% complete
99.91207502930833% complete
99.9218444704963% complete
99.93161391168425% complete
99.94138335287221% complete
99.95115279406018% complete
99.96092223524813% complete
99.9706916764361% complete
99.98046111762407% complete
99.99023055881203% complete


In [13]:
column_names = ['term', 'idf']
df_df = pd.DataFrame(columns = column_names)
element = 'word'
df_val = 0.034
df2 = pd.DataFrame([[element, df_val]], columns = column_names)
df_df = df_df.append(df2)
df_df

,term,idf
0,word,0.034


In [14]:
df3 = pd.DataFrame([['new_word', 0.5]], columns = column_names)
df_df = df_df.append(df3)
df_df

# full_comments_df['tfidf'] = tfidf_on_dataset(full_topics_df, full_comments_df)
# full_comments_df

,term,idf
0,word,0.034
0,new_word,0.500


In [87]:
full_comments_df.to_csv("files/compiled_comments_w_tfidf.csv", index=False)

In [9]:
import pandas as pd
comments_df = pd.read_csv('files/compiled_comments_2_20_2021.csv')
comments_df

,action,content,author,details,submissionId,commentId,WordScore,WholeScore,tfidf,contains_url,text_without_url
0,NaN,So what are the implications here? Does it onl...,Cody_Fox23,NaN,4op948,d4eictg,0.000000,0.849655,0.001573,False,So what are the implications here? Does it onl...
1,NaN,Sadly this isn't new. Police officers use many...,DrFrenchman,NaN,4sef35,d58ts90,0.000000,0.900283,0.255802,False,Sadly this isn't new. Police officers use many...
2,NaN,What's disturbing about this is that our gover...,bbakks,NaN,4sef35,d58y081,-0.038865,0.869078,0.000000,False,What's disturbing about this is that our gover...
3,NaN,What I find really concerning is the horrible ...,poliscijunki,NaN,4sef35,d5919n8,0.000000,0.898426,0.000000,True,What I find really concerning is the horrible ...
4,NaN,This subject might have legs but this article ...,interweb1,NaN,64zsim,dg6l969,0.000000,0.850127,0.000000,False,This subject might have legs but this article ...
...,...,...,...,...,...,...,...,...,...,...,...
11443,NaN,"Yes, while in East Baghdad my platoons mission...",CapitalCockroach,NaN,bav0rl,ekggrgk,1.070477,0.840028,0.000000,True,"Yes, while in East Baghdad my platoons mission..."
11444,NaN,The [definition the FBI currently uses for int...,CQME,NaN,bav0rl,ekyelps,0.941533,0.882768,0.217543,True,The [definition the FBI currently uses for int...
11445,NaN,[Yes.](https://en.m.wikipedia.org/wiki/Islamic...,Silent_As_The_Grave_,NaN,bav0rl,ekehcqg,0.217683,0.779386,0.000000,True,[Yes.] Have a look at who they are allies with...
11446,NaN,Has ANY Shia ever committed an act of terroris...,bsmdphdjd,NaN,bav0rl,ekfp4ls,1.293729,0.861529,0.000000,False,Has ANY Shia ever committed an act of terroris...


Trying to do some work on comment upvote feature in this notebook since the other one won't let me interrupt the kernel for some reason

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

import os
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET=os.getenv('CLIENT_SECRET')
APP_NAME=os.getenv('APP_NAME')
REDDIT_USERNAME=os.getenv('REDDIT_USERNAME')
REDDIT_PASSWORD=os.getenv('REDDIT_PASSWORD')

import praw
import pandas as pd
import datetime as dt

reddit = praw.Reddit(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, user_agent=APP_NAME, username=REDDIT_USERNAME, password=REDDIT_PASSWORD)

print(reddit.user.me())

%matplotlib inline

Version 7.1.0 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


mattcat26


In [ ]:
comments_df = pd.read_csv('files/compiled_comments_2_20_2021.csv')

def grab_comment_upvotes(com_df):
    comment_score_arr = []
    
    sum_upvotes_arr = []
    dit = {}
    count = 1
    for (index, action, content, author, details, submissionId, commentId, WordScore, WholeScore, tfidf, contains_url, text_without_url) in com_df.itertuples(name=None):
        comment = 'fake 627819 comment'
        try:
            comment = reddit.comment(commentId)
            comment_score_arr.append(comment.score)
        except:
            #If a comment cannot be read for some reason, it's upvote score will be zero
            comment_score_arr.append(0)
        
        if comment != 'fake 627819 comment':
            try:
                if submissionId in dit:
                    dit[submissionId] += comment.score
                else:
                    dit[submissionId] = comment.score
            except:
                if 'number_of_errors' in dit:
                    dit['number_of_errors'] += 1
                else:
                    dit['number_of_errors'] = 1
        if count % 50 == 0:
            print(str((index/(len(com_df.index)))*100) + "% complete (phase 1)")
            return comment.score
        count += 1
    
    count = 1
    for (index, action, content, author, details, submissionId, commentId, WordScore, WholeScore, tfidf, contains_url, text_without_url) in com_df.itertuples(name=None):
        try:
            sum_upvotes_arr.append(dit[submissionId])
        except:
            #If there are no upvotes on any comments or if there was an error, sum of upvotes will be zero
            #This could be confusing for analysis so it's something to keep in mind
            sum_upvotes_arr.append(0)
        if count % 50 == 0:
            print(str((index/(len(com_df.index)))*100) + "% complete (phase 2)")
        count+=1
    
    return comment_score_arr, sum_upvotes_arr, dit

comment = grab_comment_upvotes(comments_df)
# comments_df['comment_score'], comments_df['all_comments_scores'], new_dit = grab_comment_upvotes(comments_df)

In [ ]:
comment